<a href="https://colab.research.google.com/github/ProsperChuks/wine-quality-prediction/blob/main/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [49]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix

## Load DataSet

In [50]:
path = '/content/drive/MyDrive/Colab Notebooks/Wine Quality Project/winequality-red.csv'

dataframe = pd.read_csv(path, error_bad_lines=False)
dataframe

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


## Processing the data

In [51]:
correlation = dataframe.corr()['quality'].drop(['quality'])

columns_to_drop = ['citric acid', 'chlorides',
                   'residual sugar','quality',
                   'free sulfur dioxide',
                   'total sulfur dioxide',
                   'pH', 'sulphates']

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(dataframe.drop(columns_to_drop, axis=1))
features = pd.DataFrame(normalized_df, columns=dataframe.drop(columns_to_drop, axis=1).columns)
target = dataframe['quality']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=30)
x_test

,fixed acidity,volatile acidity,density,alcohol
1147,0.477876,0.198630,0.510279,0.523077
659,0.221239,0.493151,0.508811,0.400000
871,0.203540,0.301370,0.378120,0.338462
1333,0.398230,0.448630,0.552863,0.184615
1411,0.159292,0.239726,0.457416,0.338462
...,...,...,...,...
72,0.274336,0.390411,0.442731,0.169231
754,0.283186,0.246575,0.476505,0.107692
1059,0.380531,0.246575,0.425110,0.569231
1400,0.292035,0.390411,0.450073,0.230769


## Training the data
using Linear Regression

In [52]:
model = LinearRegression().fit(x_train, y_train)
pred = model.predict(x_test)
model.score(x_test, y_test)

0.32801211414081044

In [53]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, pred)
print('%.3f'%mae, 'MAE')

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('%.3f'%rmse, 'RMSE')

from sklearn.metrics import r2_score
r2 = r2_score(y_test, pred)
print('%.3f'%r2, 'R-Squared')

0.531 MAE
0.672 RMSE
0.328 R-Squared


## Training with a classification model

### pre-processing data for classification

In [54]:
print(dataframe['quality'].value_counts())

dataframe['quality'] = dataframe['quality'].replace([3], 4)
dataframe['quality'] = dataframe['quality'].replace([8], 7)
print(dataframe['quality'].value_counts())

dataframe = dataframe.drop('residual sugar', axis=1)
X = dataframe.drop('quality', axis=1)
y = dataframe.quality

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)
y_train.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64
5    681
6    638
7    217
4     63
Name: quality, dtype: int64


5    407
6    390
7    126
4     36
Name: quality, dtype: int64

### training the data

In [59]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)

x_train_balanced = pd.DataFrame(x_train_balanced, columns=x_train.columns)
y_balanced = pd.DataFrame(y_balanced, columns=['quality'])

normalized_train = scaler.fit_transform(x_train_balanced)
normalized_train = pd.DataFrame(normalized_train, columns=x_train_balanced.columns)

normalized_test = scaler.fit_transform(x_test)
normalized_test = pd.DataFrame(normalized_test, columns=x_train_balanced.columns)

# model
model = LogisticRegression(C=1).fit(normalized_train, y_balanced)
pred = model.predict(normalized_test)
model.score(normalized_train, y_balanced)

accuracy = accuracy_score(y_test, pred)
print('%.2f'%accuracy, 'Accuracy')

precision = precision_score(y_test, pred, average='micro')
print('%.4f'%precision, 'Precision')

0.40 Accuracy
0.3953 Precision


re-defining model;</br>

pipeline


In [56]:
X = dataframe.drop(['quality'], axis=1)
y = dataframe['quality']

pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LogisticRegression(C=1))
])

_model_2 = GridSearchCV(estimator=pipe, param_grid={
    'model__C': [1,2,3,4,5,6,7,8,9,10]
}, cv=10)

_model_2.fit(X, y)
pd.DataFrame(_model_2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.076594,0.006725,0.001465,0.000196,1,{'model__C': 1},0.51875,0.60625,0.58750,0.51250,0.5500,0.68750,0.60625,0.58125,0.65625,0.578616,0.588487,0.052245,10
1,0.091388,0.009718,0.001380,0.000160,2,{'model__C': 2},0.50000,0.61250,0.58750,0.51250,0.5500,0.68750,0.61875,0.58125,0.67500,0.566038,0.589104,0.058668,9
2,0.095225,0.003936,0.001463,0.000322,3,{'model__C': 3},0.50000,0.61875,0.60000,0.52500,0.5625,0.70625,0.60625,0.58125,0.66875,0.566038,0.593479,0.058751,8
3,0.093699,0.003849,0.001463,0.000291,4,{'model__C': 4},0.50000,0.60625,0.61250,0.51875,0.5625,0.71250,0.61250,0.58750,0.66875,0.572327,0.595358,0.060373,1
4,0.096321,0.004920,0.001404,0.000138,5,{'model__C': 5},0.50000,0.60625,0.61250,0.51875,0.5625,0.70625,0.61250,0.58125,0.66875,0.572327,0.594108,0.059283,7
5,0.093109,0.003926,0.001356,0.000184,6,{'model__C': 6},0.50000,0.60625,0.61250,0.51875,0.5625,0.71250,0.61250,0.58125,0.67500,0.572327,0.595358,0.061272,3
6,0.093064,0.004218,0.001298,0.000032,7,{'model__C': 7},0.50000,0.61250,0.61250,0.51875,0.5625,0.71250,0.61250,0.58125,0.66875,0.572327,0.595358,0.060631,1
7,0.095169,0.004988,0.001397,0.000147,8,{'model__C': 8},0.50000,0.61250,0.61250,0.51875,0.5625,0.71250,0.60625,0.58125,0.66875,0.572327,0.594733,0.060483,6
8,0.091606,0.002854,0.001396,0.000342,9,{'model__C': 9},0.50000,0.61250,0.61875,0.51875,0.5625,0.71250,0.61250,0.58125,0.66250,0.572327,0.595358,0.060113,3
9,0.091944,0.005124,0.001324,0.000066,10,{'model__C': 10},0.50000,0.61250,0.61875,0.51875,0.5625,0.71250,0.61250,0.58125,0.66250,0.572327,0.595358,0.060113,3


In [61]:
pred = _model_2.predict(X)

print('%.2f'%_model_2.score(X, y), 'Model score')
accuracy = accuracy_score(y, pred)
print('%.2f'%accuracy, 'Accuracy')

rmse = np.sqrt(mean_squared_error(y, pred))
print('%.3f'%rmse, 'RMSE')

precision = precision_score(y, pred, average='micro')
print('%.4f'%precision, 'Precision')

0.61 Model score
0.61 Accuracy
0.675 RMSE
0.6098 Precision


### Under-fitting problem

*   will add more data and extract the testing set from there making the current dataset the train set

model performed poorly on the train_test_split dataset; now trained the model on the main dataset as a whole: the model performed better